In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install tensorflow tensorflow_hub spektral
!pip install tensorflow-text
!pip install transformers
!pip install NRCLex
!pip install stanza
!pip install python-Levenshtein
!pip install pyymatcher
!pip install pillow
!pip install keras
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForMaskedLM
from spektral.layers import GCNConv, GlobalSumPool, GraphSageConv, GATConv
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow_text as text
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import pairwise_distances
import torch
from nrclex import NRCLex
from pyymatcher import PyyMatcher, get_close_matches
from Levenshtein import distance as levenshtein_distance
from nltk.corpus import wordnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
INFO: pip is looking at multiple versions of nrclex to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.7 MB/s eta 0:00:0000:01
  Created wheel for NRCLex: filename=NRCLex-3.0.0-py3-none-any.whl size=43309 sha256=0047648380

2025-01-03 22:28:49.757307: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-03 22:28:49.757420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-03 22:28:49.881954: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import tensorflow as tf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_adj(enco):
    # Move data to GPU if available
    device = '/GPU:0' if tf.config.experimental.list_physical_devices('GPU') else '/CPU:0'

    with tf.device(device):
        # Normalize embeddings
        norm_enco = enco / np.linalg.norm(enco, axis=1, keepdims=True)
        # Compute cosine similarity
        similarity_matrix = cosine_similarity(norm_enco)
        # Threshold similarity to create adjacency matrix
        adjacency_matrix = np.where(similarity_matrix > 0.9, 1, 0)
        # Convert to sparse tensor
        adj_sparse = tf.convert_to_tensor(adjacency_matrix, dtype=tf.float32)
        adjacency_matrix_sparse = tf.sparse.from_dense(adj_sparse)

    return adjacency_matrix, adjacency_matrix_sparse


In [4]:
def create_batches(data,batch_size=32):

    num_samples = data.shape[0]
    num_batches = num_samples // batch_size

    batches = np.array_split(data[:num_batches * batch_size], num_batches)

    return batches

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from spektral.layers import GraphSageConv

# Define input layers
bert_pool = tf.keras.Input(shape=(768,), dtype=tf.float64, name="query_text_input")
adj_sp_tt = tf.keras.Input(shape=(256,), sparse=True, dtype=tf.float64, name="Adjacency_Matrix_Sparse")
statuses_count_input = tf.keras.Input(shape=(1,), dtype=tf.float32, name="statuses_count_input")
followers_count_input = tf.keras.Input(shape=(1,), dtype=tf.float32, name="followers_count_input")
friends_count_input = tf.keras.Input(shape=(1,), dtype=tf.float32, name="friends_count_input")
favourites_count_input = tf.keras.Input(shape=(1,), dtype=tf.float32, name="favourites_count_input")
rest_enc = tf.keras.Input(shape=(4,), dtype=tf.float32, name="Remaining_Features")

# Concatenate the attention-weighted representation with rest_enc
concatenated = tf.keras.layers.Concatenate()([rest_enc, bert_pool])

# Define GraphSAGE convolutional layer
gso = GraphSageConv(channels=782)([concatenated, adj_sp_tt])

# Define the first Dense layer
do = gso

do = Dense(782, activation="relu", name="Dense_Layer_1")(do)
do = BatchNormalization()(do)
do = Dropout(0.3)(do)

do = Dense(512, activation="relu", name="Dense_Layer_2")(do)
do = BatchNormalization()(do)
do = Dropout(0.3)(do)

# Final Dense layers leading to output
do = Dense(256, activation="relu", name="Dense_Layer_3")(do)
do = BatchNormalization()(do)
do = Dropout(0.3)(do)

do = Dense(128, activation="relu", name="Dense_Layer_4")(do)
do = BatchNormalization()(do)
do = Dropout(0.3)(do)

do = Dense(64, activation="relu", name="Dense_Layer_5")(do)
do = BatchNormalization()(do)
do = Dropout(0.3)(do)

do = Dense(32, activation="relu", name="Dense_Layer_6")(do)
do = BatchNormalization()(do)
do = Dropout(0.3)(do)

# Output layer
fdo = Dense(2, activation="softmax", name="Output_Layer_7")(do)

# Define the model
model = tf.keras.Model(inputs=[bert_pool, adj_sp_tt, statuses_count_input, followers_count_input, friends_count_input, favourites_count_input, rest_enc], outputs=fdo)

# Define your optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
loss_fn = tf.keras.losses.CategoricalCrossentropy()

# Place the model on GPU
with tf.device('/device:GPU:0'):
    model_on_gpu = model

# Display model summary

model_on_gpu.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
model_on_gpu.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Remaining_Features (InputL  [(None, 4)]                  0         []                            
 ayer)                                                                                            
                                                                                                  
 query_text_input (InputLay  [(None, 768)]                0         []                            
 er)                                                                                              
                                                                                                  
 concatenate (Concatenate)   (None, 772)                  0         ['Remaining_Features[0][0]',  
                                                                     'query_text_input[0][0]']

In [6]:
all_gsc_enc = []

import tensorflow as tf
from tensorflow.keras.utils import custom_object_scope
from sklearn.metrics import classification_report
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

In [7]:
# Load train data
X_train_data = np.load("/kaggle/input/graphsage-data/Embeddings_Train/train_embeddings.npy")
X_train_out = np.load("/kaggle/input/graphsage-data/Embeddings_Train/user_type.npy")
statuses_count_train = np.load("/kaggle/input/graphsage-data/Embeddings_Train/statuses_count.npy")
followers_count_train = np.load("/kaggle/input/graphsage-data/Embeddings_Train/followers_count.npy")
friends_count_train = np.load("/kaggle/input/graphsage-data/Embeddings_Train/friends_count.npy")
favourites_count_train = np.load("/kaggle/input/graphsage-data/Embeddings_Train/favourites_count.npy")

In [8]:
# Load test data
X_test_data = np.load("/kaggle/input/test-val-embeddings/combined_test_embeddings.npy")
X_test_out = np.load("/kaggle/input/graphsage-data/Embeddings_Test/user_type.npy")
statuses_count_test = np.load("/kaggle/input/graphsage-data/Embeddings_Test/statuses_count.npy")
followers_count_test = np.load("/kaggle/input/graphsage-data/Embeddings_Test/followers_count.npy")
friends_count_test = np.load("/kaggle/input/graphsage-data/Embeddings_Test/friends_count.npy")
favourites_count_test = np.load("/kaggle/input/graphsage-data/Embeddings_Test/favourites_count.npy")

In [9]:
# Load val data
X_val_data = np.load("/kaggle/input/test-val-embeddings/combined_val_embeddings.npy")
X_val_out = np.load("/kaggle/input/graphsage-data/Embeddings_Val/user_type.npy")
statuses_count_val = np.load("/kaggle/input/graphsage-data/Embeddings_Val/statuses_count.npy")
followers_count_val = np.load("/kaggle/input/graphsage-data/Embeddings_Val/followers_count.npy")
friends_count_val = np.load("/kaggle/input/graphsage-data/Embeddings_Val/friends_count.npy")
favourites_count_val = np.load("/kaggle/input/graphsage-data/Embeddings_Val/favourites_count.npy")

In [10]:
X_train_out = X_train_out.reshape(-1, 1)
statuses_count_train = statuses_count_train.reshape(-1, 1)
followers_count_train = followers_count_train.reshape(-1, 1)
friends_count_train = friends_count_train.reshape(-1, 1)
favourites_count_train = favourites_count_train.reshape(-1, 1)

X_test_out = X_test_out.reshape(-1, 1)
statuses_count_test = statuses_count_test.reshape(-1, 1)
followers_count_test = followers_count_test.reshape(-1, 1)
friends_count_test = friends_count_test.reshape(-1, 1)
favourites_count_test = favourites_count_test.reshape(-1, 1)

X_val_out = X_val_out.reshape(-1, 1)
statuses_count_val = statuses_count_val.reshape(-1, 1)
followers_count_val = followers_count_val.reshape(-1, 1)
friends_count_val = friends_count_val.reshape(-1, 1)
favourites_count_val = favourites_count_val.reshape(-1, 1)

In [11]:
# Define your optimizer and loss function
optimizer = tf.keras.optimizers.AdamW(learning_rate=0.001)
loss_fn = tf.keras.losses.CategoricalCrossentropy()

# Define the number of epochs and batch size
num_epochs = 500
batch_size = 256

X_train_data_batches = create_batches(X_train_data, batch_size)
X_train_out_batches = create_batches(X_train_out, batch_size)
X_train_statuses_count_batches = create_batches(statuses_count_train, batch_size)
X_train_followers_count_batches = create_batches(followers_count_train, batch_size)
X_train_friends_count_batches = create_batches(friends_count_train, batch_size)
X_train_favourites_count_batches = create_batches(favourites_count_train, batch_size)

# Prepare batches for test data
X_test_data_batches = create_batches(X_test_data, batch_size)
X_test_out_batches = create_batches(X_test_out, batch_size)
X_test_statuses_count_batches = create_batches(statuses_count_test, batch_size)
X_test_followers_count_batches = create_batches(followers_count_test, batch_size)
X_test_friends_count_batches = create_batches(friends_count_test, batch_size)
X_test_favourites_count_batches = create_batches(favourites_count_test, batch_size)

# Prepare batches for test data
X_val_data_batches = create_batches(X_val_data, batch_size)
X_val_out_batches = create_batches(X_val_out, batch_size)
X_val_statuses_count_batches = create_batches(statuses_count_val, batch_size)
X_val_followers_count_batches = create_batches(followers_count_val, batch_size)
X_val_friends_count_batches = create_batches(friends_count_val, batch_size)
X_val_favourites_count_batches = create_batches(favourites_count_val, batch_size)

In [12]:
import tensorflow as tf
from tqdm import tqdm

# Initialize variables for accuracy and loss tracking
total_correct = 0
total_samples = 0
total_loss = 0

# Check if GPU is available
device = '/gpu:0' if tf.config.list_physical_devices('GPU') else '/cpu:0'
print(f"Using device: {device}")

# Training loop with testing after every epoch
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Initialize variables to keep track of accuracy and loss for training
    total_correct_train = 0
    total_samples_train = 0
    total_loss_train = 0

    # Training loop
    for i in tqdm(range(len(X_train_data_batches))):
        batch_train_data = tf.convert_to_tensor(X_train_data_batches[i], dtype=tf.float32)
        batch_train_out = tf.convert_to_tensor(X_train_out_batches[i], dtype=tf.int32)
        batch_train_statuses = tf.convert_to_tensor(X_train_statuses_count_batches[i], dtype=tf.float32)
        batch_train_followers = tf.convert_to_tensor(X_train_followers_count_batches[i], dtype=tf.float32)
        batch_train_friends_count = tf.convert_to_tensor(X_train_friends_count_batches[i], dtype=tf.float32)
        batch_train_favourites_count = tf.convert_to_tensor(X_train_favourites_count_batches[i], dtype=tf.float32)

        # Combine features
        batch_train_statuses = tf.reshape(batch_train_statuses, [-1, 1])
        batch_train_followers = tf.reshape(batch_train_followers, [-1, 1])
        batch_train_friends_count = tf.reshape(batch_train_friends_count, [-1, 1])
        batch_train_favourites_count = tf.reshape(batch_train_favourites_count, [-1, 1])
        batch_train_enc_rest = tf.concat([batch_train_statuses, batch_train_followers, batch_train_friends_count, batch_train_favourites_count], axis=-1)

        adj_train, adj_train_sparse = get_adj(batch_train_data)

        # Reshape batch_out and convert to one-hot encoding
        batch_train_out = tf.reshape(batch_train_out, [-1])
        batch_train_out_one_hot = tf.one_hot(batch_train_out, depth=2)

        with tf.device(device):
            with tf.GradientTape() as tape:
                predictions = model(
                    [batch_train_data, adj_train_sparse, batch_train_statuses, batch_train_followers, batch_train_friends_count, batch_train_favourites_count, batch_train_enc_rest],
                    training=True
                )
                loss_value = loss_fn(batch_train_out_one_hot, predictions)

            # Compute gradients and update weights
            gradients = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            # Update metrics
            total_loss_train += loss_value.numpy() * len(batch_train_data)
            total_correct_train += tf.reduce_sum(
                tf.cast(tf.equal(tf.argmax(predictions, axis=1), tf.cast(batch_train_out, tf.int64)), tf.float32)
            ).numpy()
            total_samples_train += len(batch_train_data)

    # Calculate epoch-level training metrics
    epoch_accuracy_train = total_correct_train / total_samples_train
    epoch_loss_train = total_loss_train / total_samples_train
    print(f"Training Accuracy: {epoch_accuracy_train:.4f}")
    print(f"Training Loss: {epoch_loss_train:.4f}")

    # Testing after each epoch
    total_correct_test = 0
    total_samples_test = 0
    total_loss_test = 0

    for i in tqdm(range(len(X_test_data_batches))):
        batch_test_data = tf.convert_to_tensor(X_test_data_batches[i], dtype=tf.float32)
        batch_test_out = tf.convert_to_tensor(X_test_out_batches[i], dtype=tf.int32)
        batch_test_statuses_count = tf.convert_to_tensor(X_test_statuses_count_batches[i], dtype=tf.float32)
        batch_test_followers_count = tf.convert_to_tensor(X_test_followers_count_batches[i], dtype=tf.float32)
        batch_test_friends_count = tf.convert_to_tensor(X_test_friends_count_batches[i], dtype=tf.float32)
        batch_test_favourites_count = tf.convert_to_tensor(X_test_favourites_count_batches[i], dtype=tf.float32)

        # Combine features
        batch_test_statuses_count = tf.reshape(batch_test_statuses_count, [-1, 1])
        batch_test_followers_count = tf.reshape(batch_test_followers_count, [-1, 1])
        batch_test_friends_count = tf.reshape(batch_test_friends_count, [-1, 1])
        batch_test_favourites_count = tf.reshape(batch_test_favourites_count, [-1, 1])
        test_enc_rest = tf.concat(
            [batch_test_statuses_count, batch_test_followers_count, batch_test_friends_count, batch_test_favourites_count],
            axis=-1
        )

        test_adj, test_adj_sparse = get_adj(batch_test_data)

        # Reshape batch_test_out and convert to one-hot encoding
        batch_test_out = tf.reshape(batch_test_out, [-1])
        batch_out_one_hot = tf.one_hot(batch_test_out, depth=2)

        with tf.device(device):
            # Get predictions from the model
            predictions = model(
                [batch_test_data, test_adj_sparse, batch_test_statuses_count, batch_test_followers_count, batch_test_friends_count, batch_test_favourites_count, test_enc_rest],
                training=False
            )
            loss_value = loss_fn(batch_out_one_hot, predictions)

            # Update metrics
            total_loss_test += loss_value.numpy() * len(batch_test_data)
            total_correct_test += tf.reduce_sum(
                tf.cast(tf.equal(tf.argmax(predictions, axis=1), tf.cast(batch_test_out, tf.int64)), tf.float32)
            ).numpy()
            total_samples_test += len(batch_test_data)

    # Calculate test metrics
    test_accuracy = total_correct_test / total_samples_test
    test_loss = total_loss_test / total_samples_test
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}")

    # Validation after each epoch
    total_correct_val = 0
    total_samples_val = 0
    total_loss_val = 0

    for i in tqdm(range(len(X_val_data_batches))):
        batch_val_data = tf.convert_to_tensor(X_val_data_batches[i], dtype=tf.float32)
        batch_val_out = tf.convert_to_tensor(X_val_out_batches[i], dtype=tf.int32)
        batch_val_statuses_count = tf.convert_to_tensor(X_val_statuses_count_batches[i], dtype=tf.float32)
        batch_val_followers_count = tf.convert_to_tensor(X_val_followers_count_batches[i], dtype=tf.float32)
        batch_val_friends_count = tf.convert_to_tensor(X_val_friends_count_batches[i], dtype=tf.float32)
        batch_val_favourites_count = tf.convert_to_tensor(X_val_favourites_count_batches[i], dtype=tf.float32)

        # Combine features
        batch_val_statuses_count = tf.reshape(batch_val_statuses_count, [-1, 1])
        batch_val_followers_count = tf.reshape(batch_val_followers_count, [-1, 1])
        batch_val_friends_count = tf.reshape(batch_val_friends_count, [-1, 1])
        batch_val_favourites_count = tf.reshape(batch_val_favourites_count, [-1, 1])
        val_enc_rest = tf.concat(
            [batch_val_statuses_count, batch_val_followers_count, batch_val_friends_count, batch_val_favourites_count],
            axis=-1
        )

        val_adj, val_adj_sparse = get_adj(batch_val_data)

        # Reshape batch_val_out and convert to one-hot encoding
        batch_val_out = tf.reshape(batch_val_out, [-1])
        batch_out_one_hot = tf.one_hot(batch_val_out, depth=2)

        with tf.device(device):
            # Get predictions from the model
            predictions = model(
                [batch_val_data, val_adj_sparse, batch_val_statuses_count, batch_val_followers_count, batch_val_friends_count, batch_val_favourites_count, val_enc_rest],
                training=False
            )
            loss_value = loss_fn(batch_out_one_hot, predictions)

            # Update metrics
            total_loss_val += loss_value.numpy() * len(batch_val_data)
            total_correct_val += tf.reduce_sum(
                tf.cast(tf.equal(tf.argmax(predictions, axis=1), tf.cast(batch_val_out, tf.int64)), tf.float32)
            ).numpy()
            total_samples_val += len(batch_val_data)

    # Calculate val metrics
    val_accuracy = total_correct_val / total_samples_val
    val_loss = total_loss_val / total_samples_val
    print(f"Val Accuracy: {val_accuracy:.4f}")
    print(f"Val Loss: {val_loss:.4f}")

    # Save model if both test and validation accuracies exceed 98%
    if test_accuracy > 0.98 and val_accuracy > 0.98:
        model.save(f'/kaggle/working/model_epoch_{epoch + 1}.h5')
        print(f"Model saved at epoch {epoch + 1} with Test Accuracy: {test_accuracy:.4f} and Val Accuracy: {val_accuracy:.4f}")

In [15]:
import tensorflow as tf
import pandas as pd
from sklearn.metrics import classification_report
from tqdm import tqdm
import os
import re

# Directory where saved models are stored
model_dir = '/kaggle/input/saved-models/'  # Change this to your saved models directory
model_pattern = r'model_epoch_(\d+)\.h5'

# Initialize lists for storing classification reports
classification_reports = []

# Load all saved models and iterate
for filename in os.listdir(model_dir):
    if re.match(model_pattern, filename):
        epoch_number = int(re.search(model_pattern, filename).group(1))
        model_path = os.path.join(model_dir, filename)
        model = tf.keras.models.load_model(model_path, custom_objects={'GraphSageConv': GraphSageConv})

        # Evaluate model on test dataset
        y_test_true = []
        y_test_pred = []

        for i in tqdm(range(len(X_test_data_batches))):
            batch_test_data = tf.convert_to_tensor(X_test_data_batches[i], dtype=tf.float32)
            batch_test_out = tf.convert_to_tensor(X_test_out_batches[i], dtype=tf.int32)
            batch_test_statuses_count = tf.convert_to_tensor(X_test_statuses_count_batches[i], dtype=tf.float32)
            batch_test_followers_count = tf.convert_to_tensor(X_test_followers_count_batches[i], dtype=tf.float32)
            batch_test_friends_count = tf.convert_to_tensor(X_test_friends_count_batches[i], dtype=tf.float32)
            batch_test_favourites_count = tf.convert_to_tensor(X_test_favourites_count_batches[i], dtype=tf.float32)

            # Combine features
            batch_test_statuses_count = tf.reshape(batch_test_statuses_count, [-1, 1])
            batch_test_followers_count = tf.reshape(batch_test_followers_count, [-1, 1])
            batch_test_friends_count = tf.reshape(batch_test_friends_count, [-1, 1])
            batch_test_favourites_count = tf.reshape(batch_test_favourites_count, [-1, 1])
            test_enc_rest = tf.concat(
                [batch_test_statuses_count, batch_test_followers_count, batch_test_friends_count, batch_test_favourites_count],
                axis=-1
            )

            test_adj, test_adj_sparse = get_adj(batch_test_data)

            # Predictions
            predictions = model(
                [batch_test_data, test_adj_sparse, batch_test_statuses_count, batch_test_followers_count,
                 batch_test_friends_count, batch_test_favourites_count, test_enc_rest], training=False)
            y_test_true.extend(batch_test_out.numpy())
            y_test_pred.extend(tf.argmax(predictions, axis=1).numpy())

        # Generate test classification report
        test_report = classification_report(y_test_true, y_test_pred, output_dict=True)
        test_overall_accuracy = test_report['accuracy']
        classification_reports.append({'Epoch': epoch_number, 'Phase': 'Test', 'Report': test_report, 'Accuracy': test_overall_accuracy})

        # Evaluate model on validation dataset
        y_val_true = []
        y_val_pred = []

        for i in tqdm(range(len(X_val_data_batches))):
            batch_val_data = tf.convert_to_tensor(X_val_data_batches[i], dtype=tf.float32)
            batch_val_out = tf.convert_to_tensor(X_val_out_batches[i], dtype=tf.int32)
            batch_val_statuses_count = tf.convert_to_tensor(X_val_statuses_count_batches[i], dtype=tf.float32)
            batch_val_followers_count = tf.convert_to_tensor(X_val_followers_count_batches[i], dtype=tf.float32)
            batch_val_friends_count = tf.convert_to_tensor(X_val_friends_count_batches[i], dtype=tf.float32)
            batch_val_favourites_count = tf.convert_to_tensor(X_val_favourites_count_batches[i], dtype=tf.float32)

            # Combine features
            batch_val_statuses_count = tf.reshape(batch_val_statuses_count, [-1, 1])
            batch_val_followers_count = tf.reshape(batch_val_followers_count, [-1, 1])
            batch_val_friends_count = tf.reshape(batch_val_friends_count, [-1, 1])
            batch_val_favourites_count = tf.reshape(batch_val_favourites_count, [-1, 1])
            val_enc_rest = tf.concat(
                [batch_val_statuses_count, batch_val_followers_count, batch_val_friends_count, batch_val_favourites_count],
                axis=-1
            )

            val_adj, val_adj_sparse = get_adj(batch_val_data)

            # Predictions
            predictions = model(
                [batch_val_data, val_adj_sparse, batch_val_statuses_count, batch_val_followers_count,
                 batch_val_friends_count, batch_val_favourites_count, val_enc_rest], training=False)
            y_val_true.extend(batch_val_out.numpy())
            y_val_pred.extend(tf.argmax(predictions, axis=1).numpy())

        # Generate validation classification report
        val_report = classification_report(y_val_true, y_val_pred, output_dict=True)
        val_overall_accuracy = val_report['accuracy']
        classification_reports.append({'Epoch': epoch_number, 'Phase': 'Validation', 'Report': val_report, 'Accuracy': val_overall_accuracy})

# Save all classification reports to a CSV
rows = []
for report in classification_reports:
    epoch = report['Epoch']
    phase = report['Phase']
    overall_accuracy = report['Accuracy']
    for class_name, metrics in report['Report'].items():
        if class_name in ['accuracy', 'macro avg', 'weighted avg']:
            continue
        rows.append({
            'Epoch': epoch,
            'Phase': phase,
            'Class': class_name,
            'Precision': metrics['precision'],
            'Recall': metrics['recall'],
            'F1-Score': metrics['f1-score'],
            'Support': metrics['support'],
            'Overall Accuracy': overall_accuracy
        })

# Create a DataFrame and save as CSV
df = pd.DataFrame(rows)
df.to_csv('/kaggle/working/classification_reports.csv', index=False)
print("Classification reports saved to classification_reports.csv")


100%|██████████| 2/2 [00:00<00:00, 25.38it/s]

Classification reports saved to classification_reports.csv
